In [1]:
import pandas as pd
import statsmodels.api as sm

import warnings # Suppress warnings because they are annoying
warnings.filterwarnings('ignore') 

### 1. Analysis in Belgium

In [2]:
# reading the data in Belgium
hosp = pd.read_csv('COVID19BE_HOSP.csv', encoding = 'latin_1')
cases = pd.read_csv('COVID19BE_CASES_AGESEX.csv', encoding = 'latin_1')
mort = pd.read_csv('COVID19BE_MORT.csv', encoding = 'latin_1')

In [21]:
# cleaning the hospital data
hosp_temp = hosp[hosp['DATE'] == '2020-05-01']
hosp_temp.reset_index(drop = True, inplace = True)
hosp_temp.drop(['DATE', 'TOTAL_IN', 'TOTAL_IN_ICU', 'TOTAL_IN_RESP', 'TOTAL_IN_ECMO', 'NEW_IN', 'NEW_OUT'], axis = 1, inplace = True)
hosp_temp.loc[4,'PROVINCE'] = 'Liege'
hosp_temp.to_csv('Host_temp.csv', encoding = 'utf-8')

In [4]:
# cleaning the number of cases data
cases_temp = pd.DataFrame(cases.groupby('PROVINCE')['CASES'].sum()).reset_index()
cases_temp.loc[5,'PROVINCE'] = 'Liege'

In [5]:
# cleaning the mortality data
mort_temp = pd.DataFrame(mort.groupby('REGION')['DEATHS'].sum()).reset_index()
mort_temp

,REGION,DEATHS
0,Brussels,1219
1,Flanders,3904
2,Wallonia,2721


In [6]:
# combining all three data
data_temp = cases_temp.merge(hosp_temp, left_on = 'PROVINCE', right_on = 'PROVINCE')
data_temp = data_temp.groupby('REGION')[['CASES', 'NR_REPORTING']].sum().reset_index()
data = data_temp.merge(mort_temp, left_on = 'REGION', right_on = 'REGION')
data

,REGION,CASES,NR_REPORTING,DEATHS
0,Brussels,5055,15,1219
1,Flanders,27331,52,3904
2,Wallonia,16132,37,2721


In [7]:
# checking the correlations
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,CASES,NR_REPORTING,DEATHS
CASES,1.00,0.99,1.00
NR_REPORTING,0.99,1.00,1.00
DEATHS,1.00,1.00,1.00


In [8]:
# linear regression
y = data['DEATHS']
X = data['NR_REPORTING']
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 DEATHS   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     614.7
Date:                Sun, 03 May 2020   Prob (F-statistic):             0.0257
Time:                        14:56:34   Log-Likelihood:                -15.628
No. Observations:                   3   AIC:                             35.26
Df Residuals:                       1   BIC:                             33.45
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          110.0250    110.296      0.998      0.501   -1291.416    1511.466
NR_REPORTING    72.2493      2.914     24.794      0.026      35.224     109.275
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.699
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.506
Skew:                          -0.670   Prob(JB):                        0.777
Kurtosis:                       1.500   Cond. No.                         94.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### 2. Analysis by countries

In [9]:
# reading data by countries in the EU
HospBeds = pd.read_csv('DP_LIVE_HOSPBEDS.csv') # per 1000
Doctors = pd.read_csv('DP_LIVE_DOCTORS.csv') # per 1000
Nurses = pd.read_csv('DP_LIVE_NURSES.csv') # per 1000
Spendings = pd.read_csv('DP_LIVE_SPENDINGS.csv') # USD per capita
corona = pd.read_csv('05-02-2020.csv')
pop = pd.read_csv('population.csv')
corona

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-05-03 02:32:28,34.223334,-82.461707,31,0,0,31,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-05-03 02:32:28,30.295065,-92.414197,133,10,0,123,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-05-03 02:32:28,37.767072,-75.632346,353,5,0,348,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-05-03 02:32:28,43.452658,-116.241552,705,17,0,688,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-05-03 02:32:28,41.330756,-94.471059,1,0,0,1,"Adair, Iowa, US"
...,...,...,...,...,...,...,...,...,...,...,...,...
3187,NaN,NaN,NaN,West Bank and Gaza,2020-05-03 02:32:28,31.952200,35.233200,353,2,76,275,West Bank and Gaza
3188,NaN,NaN,NaN,Western Sahara,2020-05-03 02:32:28,24.215500,-12.885800,6,0,5,1,Western Sahara
3189,NaN,NaN,NaN,Yemen,2020-05-03 02:32:28,15.552727,48.516388,10,2,1,7,Yemen
3190,NaN,NaN,NaN,Zambia,2020-05-03 02:32:28,-13.133897,27.849332,119,3,75,41,Zambia


In [10]:
# grouping corona data by countries
# adding the new column represents the death ratio to the number of cases.
corona = corona.groupby('Country_Region')[['Confirmed', 'Deaths', 'Recovered', 'Active']].sum().reset_index()
corona['Deaths_Rate'] = corona['Deaths'] / corona['Confirmed']
corona

,Country_Region,Confirmed,Deaths,Recovered,Active,Deaths_Rate
0,Afghanistan,2469,72,331,2066,0.029162
1,Albania,789,31,519,239,0.039290
2,Algeria,4295,459,1872,1964,0.106868
3,Andorra,747,44,472,231,0.058902
4,Angola,35,2,11,22,0.057143
...,...,...,...,...,...,...
182,West Bank and Gaza,353,2,76,275,0.005666
183,Western Sahara,6,0,5,1,0.000000
184,Yemen,10,2,1,7,0.200000
185,Zambia,119,3,75,41,0.025210


In [11]:
# cleaning the population data
pop = pop[['Country Name', 'Country Code', '2018']]
pop = pop.rename(columns = {'2018': 'population'})
pop

,Country Name,Country Code,population
0,Aruba,ABW,105845.0
1,Afghanistan,AFG,37172386.0
2,Angola,AGO,30809762.0
3,Albania,ALB,2866376.0
4,Andorra,AND,77006.0
...,...,...,...
259,Kosovo,XKX,1845300.0
260,"Yemen, Rep.",YEM,28498687.0
261,South Africa,ZAF,57779622.0
262,Zambia,ZMB,17351822.0


In [12]:
# concatinating and grouping all data
data = pd.concat([HospBeds, Doctors, Nurses, Spendings])
data = data.groupby(['LOCATION', 'INDICATOR'])['Value'].mean().reset_index()
data

,LOCATION,INDICATOR,Value
0,AUT,HEALTHEXP,3458.339250
1,AUT,HOSPITALBED,7.443333
2,AUT,MEDICALDOC,5.133333
3,AUT,NURSE,6.806667
4,BEL,HEALTHEXP,3161.437417
...,...,...,...
85,SVN,NURSE,9.450000
86,SWE,HEALTHEXP,3464.619500
87,SWE,HOSPITALBED,2.333333
88,SWE,MEDICALDOC,4.195000


In [13]:
# pivoting data with indicator columns
data = data.pivot_table(index = ['LOCATION'],
                        columns = 'INDICATOR',
                        values = 'Value')
data = data.dropna()
data = data.reset_index()
data

INDICATOR,LOCATION,HEALTHEXP,HOSPITALBED,MEDICALDOC,NURSE
0,AUT,3458.339250,7.443333,5.133333,6.806667
1,BEL,3161.437417,5.722500,3.056667,10.895000
2,DEU,3779.259083,8.063333,4.193333,12.806667
3,DNK,3295.351333,2.560000,3.965000,9.935000
4,ESP,2112.833750,2.973333,3.850000,5.513333
5,EST,1372.171083,4.803333,3.446667,6.100000
6,FRA,3235.884333,6.056667,3.357500,10.355000
7,GBR,2591.467417,2.573333,2.802500,7.855000
8,HUN,1316.661917,7.003333,3.210000,6.473333
9,IRL,3053.201333,2.950000,3.012500,12.023333


In [14]:
# merging the data with the population data
data = data.merge(pop, left_on = 'LOCATION', right_on = 'Country Code')
data.drop(['Country Code'], axis = 1, inplace = True)
data

,LOCATION,HEALTHEXP,HOSPITALBED,MEDICALDOC,NURSE,Country Name,population
0,AUT,3458.339250,7.443333,5.133333,6.806667,Austria,8840521.0
1,BEL,3161.437417,5.722500,3.056667,10.895000,Belgium,11433256.0
2,DEU,3779.259083,8.063333,4.193333,12.806667,Germany,82905782.0
3,DNK,3295.351333,2.560000,3.965000,9.935000,Denmark,5793636.0
4,ESP,2112.833750,2.973333,3.850000,5.513333,Spain,46796540.0
5,EST,1372.171083,4.803333,3.446667,6.100000,Estonia,1321977.0
6,FRA,3235.884333,6.056667,3.357500,10.355000,France,66977107.0
7,GBR,2591.467417,2.573333,2.802500,7.855000,United Kingdom,66460344.0
8,HUN,1316.661917,7.003333,3.210000,6.473333,Hungary,9775564.0
9,IRL,3053.201333,2.950000,3.012500,12.023333,Ireland,4867309.0


In [15]:
# merging the data with the corona data
data = data.merge(corona, left_on = 'Country Name', right_on = 'Country_Region')
data.drop(['Country_Region'], axis = 1, inplace = True)

In [16]:
# calculating the numbers per 1000
data['Confirmed_1000'] = data['Confirmed'] / data['population'] * 1000
data['Deaths_1000'] = data['Deaths'] / data['population'] * 1000
data['Recovered_1000'] = data['Recovered'] / data['population'] * 1000
data

,LOCATION,HEALTHEXP,HOSPITALBED,MEDICALDOC,NURSE,Country Name,population,Confirmed,Deaths,Recovered,Active,Deaths_Rate,Confirmed_1000,Deaths_1000,Recovered_1000
0,AUT,3458.339250,7.443333,5.133333,6.806667,Austria,8840521.0,15558,596,13180,1782,0.038308,1.759851,0.067417,1.490862
1,BEL,3161.437417,5.722500,3.056667,10.895000,Belgium,11433256.0,49517,7765,12211,29541,0.156815,4.330962,0.679159,1.068025
2,DEU,3779.259083,8.063333,4.193333,12.806667,Germany,82905782.0,164967,6812,129000,29155,0.041293,1.989813,0.082166,1.555983
3,DNK,3295.351333,2.560000,3.965000,9.935000,Denmark,5793636.0,9605,475,7084,2046,0.049453,1.657854,0.081987,1.222721
4,ESP,2112.833750,2.973333,3.850000,5.513333,Spain,46796540.0,216582,25100,117248,74234,0.115891,4.628163,0.536364,2.505484
5,EST,1372.171083,4.803333,3.446667,6.100000,Estonia,1321977.0,1699,53,256,1390,0.031195,1.285196,0.040091,0.193649
6,FRA,3235.884333,6.056667,3.357500,10.355000,France,66977107.0,168518,24763,50663,93092,0.146946,2.516054,0.369723,0.756423
7,GBR,2591.467417,2.573333,2.802500,7.855000,United Kingdom,66460344.0,183500,28205,896,154399,0.153706,2.761045,0.424388,0.013482
8,HUN,1316.661917,7.003333,3.210000,6.473333,Hungary,9775564.0,2942,335,625,1982,0.113868,0.300955,0.034269,0.063935
9,IRL,3053.201333,2.950000,3.012500,12.023333,Ireland,4867309.0,21176,1286,13386,6504,0.060729,4.350659,0.264212,2.750185


In [17]:
# checking the correlations
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,HEALTHEXP,HOSPITALBED,MEDICALDOC,NURSE,population,Confirmed,Deaths,Recovered,Active,Deaths_Rate,Confirmed_1000,Deaths_1000,Recovered_1000
HEALTHEXP,1.00,-0.13,0.34,0.79,0.25,0.24,0.14,0.28,0.15,0.20,0.55,0.35,0.46
HOSPITALBED,-0.13,1.00,0.16,-0.09,0.06,-0.21,-0.35,0.07,-0.36,-0.34,-0.42,-0.44,-0.14
MEDICALDOC,0.34,0.16,1.00,0.02,0.00,0.09,-0.03,0.31,-0.11,-0.16,-0.11,-0.10,-0.01
NURSE,0.79,-0.09,0.02,1.00,0.06,-0.00,-0.11,0.10,-0.07,0.07,0.46,0.17,0.42
population,0.25,0.06,0.00,0.06,1.00,0.89,0.78,0.73,0.78,0.47,0.13,0.40,-0.06
Confirmed,0.24,-0.21,0.09,-0.00,0.89,1.00,0.94,0.79,0.87,0.60,0.37,0.66,0.08
Deaths,0.14,-0.35,-0.03,-0.11,0.78,0.94,1.00,0.56,0.96,0.74,0.38,0.74,0.00
Recovered,0.28,0.07,0.31,0.10,0.73,0.79,0.56,1.00,0.40,0.20,0.31,0.37,0.26
Active,0.15,-0.36,-0.11,-0.07,0.78,0.87,0.96,0.40,1.00,0.74,0.31,0.68,-0.09
Deaths_Rate,0.20,-0.34,-0.16,0.07,0.47,0.60,0.74,0.20,0.74,1.00,0.29,0.82,-0.22


In [18]:
# linear regression
y = data['Deaths_1000']
X = data[['HEALTHEXP', 'HOSPITALBED', 'MEDICALDOC', 'NURSE']]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Deaths_1000   R-squared:                       0.361
Model:                            OLS   Adj. R-squared:                  0.147
Method:                 Least Squares   F-statistic:                     1.691
Date:                Sun, 03 May 2020   Prob (F-statistic):              0.216
Time:                        14:56:40   Log-Likelihood:                 6.4048
No. Observations:                  17   AIC:                            -2.810
Df Residuals:                      12   BIC:                             1.356
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.5949      0.336      1.769      0.102      -0.138       1.328
HEALTHEXP       0.0002      0.000      1.684      0.118   -5.15e-05       0.000
HOSPITALBED    -0.0368      0.027     -1.359      0.199      -0.096       0.022
MEDICALDOC     -0.0914      0.085     -1.077      0.303      -0.276       0.093
NURSE          -0.0367      0.034     -1.082      0.301      -0.111       0.037
==============================================================================
Omnibus:                        4.131   Durbin-Watson:                   2.393
Prob(Omnibus):                  0.127   Jarque-Bera (JB):                2.265
Skew:                           0.876   Prob(JB):                        0.322
Kurtosis:                       3.358   Cond. No.                     1.88e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.88e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [19]:
# linear regression
y = data['Recovered_1000']
X = data[['HEALTHEXP', 'HOSPITALBED', 'MEDICALDOC', 'NURSE']]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         Recovered_1000   R-squared:                       0.246
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.9795
Date:                Sun, 03 May 2020   Prob (F-statistic):              0.455
Time:                        14:56:40   Log-Likelihood:                -27.065
No. Observations:                  17   AIC:                             64.13
Df Residuals:                      12   BIC:                             68.30
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.4530      2.409      0.188      0.854      -4.796       5.702
HEALTHEXP       0.0007      0.001      0.943      0.364      -0.001       0.002
HOSPITALBED    -0.0343      0.194     -0.177      0.862      -0.457       0.388
MEDICALDOC     -0.3227      0.608     -0.531      0.605      -1.647       1.002
NURSE           0.0293      0.243      0.120      0.906      -0.501       0.559
==============================================================================
Omnibus:                       11.033   Durbin-Watson:                   1.548
Prob(Omnibus):                  0.004   Jarque-Bera (JB):                7.858
Skew:                           1.329   Prob(JB):                       0.0197
Kurtosis:                       5.006   Cond. No.                     1.88e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.88e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""